In [2]:
import pandas as pd
data1 = pd.read_excel('please.xlsx', sheet_name='사원정보')
data2 = pd.read_excel('please.xlsx', sheet_name='프로젝트 내역')

In [3]:
embedding_data=data1[['사원번호','개인 성향']].merge(data2[['사원번호','프로젝트 설명']], on='사원번호', how='inner')

In [4]:
embedding_data

,사원번호,개인 성향,프로젝트 설명
0,1,"논리적이고 분석적인 문제 해결을 선호하며, 팀의 목표를 위해 데이터 기반 의사 결정...",다양한 판매자가 상품을 등록하고 관리할 수 있는 온라인 쇼핑몰 플랫폼 제작. 팔로우...
1,2,창의적 아이디어를 통해 새로운 방식을 시도함. 다양한 관점에서 문제를 해결하려 함.,사용자 위치에 기반한 지역 기반 커뮤니티앱 개발
2,3,협업을 중요시하고 피드백을 적극적으로 반영함. 피드백을 통한 개선을 중요시 여김.,사용자의 투자 성향에 맞춰 자산을 자동으로배분해주는 앱 개발
3,4,"안정적인 방법, 기존의 검증된 방식을 선호함. 체계적 업무 수행을 중요시 여김.",사용자의 신용 정보를 분석해 맞춤형 대출 상품을 추천하는 웹 개발
4,5,빠른 성과를 중시하고 신속한 업무 처리를 선호함. 단기적 문제 해결에 특화됨.,사용자 신체 데이터를 기반으로 한 개인 맞춤형 운동 및 식단 제공 서비스 개발
...,...,...,...
95,96,소통에 능숙하며 새로운 환경에 적응하는 것이 빠름. 외향적인 성격.,사용자 구매 데이터를 활용한 상품 추천 모델 개발
96,97,구조 중시. 명확히 역할이 나눠진 환경에서 효율성을 발휘함.,과거 주가 움직임 및 기업 재무 데이터를 분석하여 주가 예측 모델 개발
97,98,소수의 팀원들과 일하는 것을 선호. 조용하고 차분한 성격.,환자 데이터를 활용한 이상 징후 탐지 모델 개발
98,99,정해진 업무 절차를 따르는 것을 선호함. 새로운 변화에 민감함.,상품 리뷰 감정 분석을 통해 각 상품의 긍/부정 판단 모델 개발


In [5]:
import os
from openai import OpenAI
import openai
import pandas as pd
from dotenv import load_dotenv
import numpy as np

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

client = OpenAI(
    api_key=os.getenv("UPSTAGE_API_KEY"),
    base_url="https://api.upstage.ai/v1/solar"
)

In [6]:
personality_embeddings=[]
project_embeddings=[]
for i in range(len(embedding_data)):
    upstage_embedding_vector=client.embeddings.create(
        input=embedding_data['개인 성향'][i],
        model="solar-embedding-1-large-query"
    )

    personality_embeddings.append(upstage_embedding_vector.data[0].embedding)

for i in range(len(embedding_data)):
    upstage_embedding_vector=client.embeddings.create(
        input=embedding_data['프로젝트 설명'][i],
        model="solar-embedding-1-large-query"
    )

    project_embeddings.append(upstage_embedding_vector.data[0].embedding)

In [7]:
embedding_data['성향 임베딩 벡터']=personality_embeddings
embedding_data['프로젝트 임베딩 벡터']=project_embeddings

In [8]:
raw_data=embedding_data[['사원번호','개인 성향','프로젝트 설명']]
raw_data

,사원번호,개인 성향,프로젝트 설명
0,1,"논리적이고 분석적인 문제 해결을 선호하며, 팀의 목표를 위해 데이터 기반 의사 결정...",다양한 판매자가 상품을 등록하고 관리할 수 있는 온라인 쇼핑몰 플랫폼 제작. 팔로우...
1,2,창의적 아이디어를 통해 새로운 방식을 시도함. 다양한 관점에서 문제를 해결하려 함.,사용자 위치에 기반한 지역 기반 커뮤니티앱 개발
2,3,협업을 중요시하고 피드백을 적극적으로 반영함. 피드백을 통한 개선을 중요시 여김.,사용자의 투자 성향에 맞춰 자산을 자동으로배분해주는 앱 개발
3,4,"안정적인 방법, 기존의 검증된 방식을 선호함. 체계적 업무 수행을 중요시 여김.",사용자의 신용 정보를 분석해 맞춤형 대출 상품을 추천하는 웹 개발
4,5,빠른 성과를 중시하고 신속한 업무 처리를 선호함. 단기적 문제 해결에 특화됨.,사용자 신체 데이터를 기반으로 한 개인 맞춤형 운동 및 식단 제공 서비스 개발
...,...,...,...
95,96,소통에 능숙하며 새로운 환경에 적응하는 것이 빠름. 외향적인 성격.,사용자 구매 데이터를 활용한 상품 추천 모델 개발
96,97,구조 중시. 명확히 역할이 나눠진 환경에서 효율성을 발휘함.,과거 주가 움직임 및 기업 재무 데이터를 분석하여 주가 예측 모델 개발
97,98,소수의 팀원들과 일하는 것을 선호. 조용하고 차분한 성격.,환자 데이터를 활용한 이상 징후 탐지 모델 개발
98,99,정해진 업무 절차를 따르는 것을 선호함. 새로운 변화에 민감함.,상품 리뷰 감정 분석을 통해 각 상품의 긍/부정 판단 모델 개발


In [9]:
from langchain_community.document_loaders import DataFrameLoader

# 데이터 프레임 로더 설정, 페이지 내용 컬럼 지정
loader = DataFrameLoader(raw_data, page_content_column="개인 성향")

# 문서 로드
docs = loader.load()

# 데이터 출력
print(docs[0].page_content)

# 메타데이터 출력
print(docs[0].metadata)


논리적이고 분석적인 문제 해결을 선호하며, 팀의 목표를 위해 데이터 기반 의사 결정을 중시함. 독립적으로 일하는 것을 선호.
{'사원번호': 1, '프로젝트 설명': '다양한 판매자가 상품을 등록하고 관리할 수 있는 온라인 쇼핑몰 플랫폼 제작. 팔로우 기반 소통하는 블로그 사이트 개발'}


In [11]:
import chromadb

client.delete_collection(name="personality")

#client = chromadb.PersistentClient(path="/personality")
#collection = client.create_collection(name="personality")

In [10]:
import chromadb
from langchain_upstage import UpstageEmbeddings
from langchain_chroma import Chroma

#client = chromadb.PersistentClient(path="/personality")
#collection = client.create_collection(name="personality")


embedding = UpstageEmbeddings(model="solar-embedding-1-large")

#vectorstore = Chroma.from_documents(documents=docs, embedding=embedding, collection_name="personality", client=client, persist_directory='./personality')
vectorstore = Chroma.from_documents(documents=docs, embedding=embedding, collection_name="personality", persist_directory='./personality')


In [ ]:
from langchain_community.document_loaders import DataFrameLoader

# 데이터 프레임 로더 설정, 페이지 내용 컬럼 지정
loader = DataFrameLoader(raw_data, page_content_column="프로젝트 설명")

# 문서 로드
docs = loader.load()

# 데이터 출력
print(docs[0].page_content)

# 메타데이터 출력
print(docs[0].metadata)

embedding = UpstageEmbeddings(model="solar-embedding-1-large")

vectorstore2 = Chroma.from_documents(documents=docs, embedding=embedding, collection_name="project", persist_directory='./project')

다양한 판매자가 상품을 등록하고 관리할 수 있는 온라인 쇼핑몰 플랫폼 제작. 팔로우 기반 소통하는 블로그 사이트 개발
{'사원번호': 1, '개인 성향': '논리적이고 분석적인 문제 해결을 선호하며, 팀의 목표를 위해 데이터 기반 의사 결정을 중시함. 독립적으로 일하는 것을 선호.'}
Documents successfully saved in ChromaDB!


In [13]:
query='논리적 문제해결 선호'
retrieved_docs = vectorstore.similarity_search(query, k=100)

In [20]:
retrieved_docs

[Document(metadata={'사원번호': 65, '프로젝트 설명': '환자의 데이터를 실시간 모니터링하고, 이상 징후를 관리하는 시스템 개발'}, page_content='데이터 기반의 의사결정을 중시함. 독립적으로 일하는 것을 선호'),
 Document(metadata={'사원번호': 65, '프로젝트 설명': '환자의 데이터를 실시간 모니터링하고, 이상 징후를 관리하는 시스템 개발'}, page_content='데이터 기반의 의사결정을 중시함. 독립적으로 일하는 것을 선호'),
 Document(metadata={'사원번호': 83, '프로젝트 설명': '온라인 교육 플랫폼 제작 총괄 및 제작 이후 성과 분석 및 개선'}, page_content='계획적인 업무 방식을 중시함. 일정 루틴 속에서 일을 진행하는 것을 선호'),
 Document(metadata={'사원번호': 83, '프로젝트 설명': '온라인 교육 플랫폼 제작 총괄 및 제작 이후 성과 분석 및 개선'}, page_content='계획적인 업무 방식을 중시함. 일정 루틴 속에서 일을 진행하는 것을 선호'),
 Document(metadata={'사원번호': 37, '프로젝트 설명': '게시물 작성 및 댓글을 통해 상호작용할 수 있는 SNS 개발'}, page_content='논리적으로 문제를 해결함. 독립적으로 일하는 것을 선호하며 복잡한 문제에도 깊게 파고드는 성향.'),
 Document(metadata={'사원번호': 37, '프로젝트 설명': '게시물 작성 및 댓글을 통해 상호작용할 수 있는 SNS 개발'}, page_content='논리적으로 문제를 해결함. 독립적으로 일하는 것을 선호하며 복잡한 문제에도 깊게 파고드는 성향.'),
 Document(metadata={'사원번호': 1, '프로젝트 설명': '다양한 판매자가 상품을 등록하고 관리할 수 있는 온라인 쇼핑몰 플랫폼 제작. 팔로우 기반 소통하는 블로그 사이트 개발'}, page_content='논리적이고 분석적인

In [ ]:
retrieved_docs[0].page_content

'안전하게 온라인 거래를 할 수 있는 이커머스 거래 플랫폼 개발'

In [26]:
vectorstore2.get()

{'ids': ['007ce229-846d-4ba6-9509-e55e839ee412',
  '00a0bde2-a8e1-4176-b57b-95a8572e91c7',
  '02d874e9-d780-46db-b8d7-5ca5157375e4',
  '03bcde59-da8f-48fc-b6ba-37ca47950905',
  '05e7b1a3-cff0-4607-b018-77d01e5ac181',
  '061da279-aa7f-4f48-9a51-c7cbda5879a6',
  '06c9b053-27c4-4634-88e5-5b3fd85db905',
  '08e251c4-b1bb-4616-b372-c602e7814fe2',
  '1527cd1a-b648-4919-bf1a-17ccfdb37d1f',
  '1671af0e-95f4-4586-8803-daa6bfe8ce0b',
  '178485c6-b3e0-463a-8f38-efb1b15d9101',
  '1b313caf-b5d4-46be-9a9d-01e3cdb627f3',
  '1d91373c-b706-4045-99b3-0a9603630e9d',
  '27c4af8e-9544-4ba0-ab03-8025bc6dff47',
  '29d8c640-6521-4dd9-a0df-c059503b222a',
  '2bb637d7-e81b-44b3-8522-8f90dfcdf745',
  '2c3cfe97-5c28-4f7a-b45f-0fd650fcb71d',
  '2de4fd73-e8e4-423b-aea0-1f485aa22c1d',
  '2e9b6b6d-452f-472b-ab49-ba4e2fefc45b',
  '2eb11e71-0501-4f96-94a4-2f702d034c0b',
  '3835f653-e1c3-49f7-8993-8d8c45bd88d2',
  '3f23cc6a-7868-473f-96f5-8f980c390104',
  '423e9e82-6ba5-47fc-9428-ef1d52a839e4',
  '478335f3-887e-40cd-b1d3-

In [1]:
query='온라인 쇼핑몰 플랫폼 제작'
retrieved_docs = vectorstore2.similarity_search(query, k=3)

NameError: name 'vectorstore2' is not defined